# Evaluate military vehicle detections on The Search_2 dataset

This notebook provides a small benchmark to evaluate automated military vehicle detection models on a real dataset. Deep-learning object detection models can give reasonable detection performance when fine-tuned on specific datasets. However, acquiring enough data corresponding to a real military setting is a challenge, as demonstrated by this project. It is therefore important to evaluate these models in a military setting, with a target area around tens of pixels in a cluttered environment.

We propose to use [The Search 2](https://figshare.com/articles/dataset/The_Search_2_dataset/1041463) dataset for such an evaluation. The Search_2 dataset consists of 44 high-resolution digital color images of different complex natural scenes, with each scene (image) containing a single military vehicle that serves as a search target. Ground truth annotations are provided for the targets.

### Download and load the dataset

To begin, we download the dataset and load it into fiftyone.

In [2]:
from pathlib import Path
from adomvi.utils import download_and_extract

search_2_dir = Path() / "search_2"
search_2_url = "https://github.com/jonasrenault/adomvi/releases/download/v1.3.0/search_2.tar.gz"
download_and_extract(search_2_url, "search_2.tar.gz", search_2_dir)

In [3]:
from adomvi.datasets.search2 import load_search_2_dataset

dataset = load_search_2_dataset(search_2_dir / "search_2")

 100% |███████████████████| 44/44 [56.6ms elapsed, 0s remaining, 777.4 samples/s]     


We can map the labels which identify each target in the dataset to the four classes with which our model was trained.

In [6]:
label_mapping = {
    "M60": "AFV",
    "M3": "AFV",
    "M1": "AFV",
    "T72": "AFV",
    "HVS": "LAV",
    "HVT": "LAV",
    "BMP": "APC",
    "BTR": "APC",
    "M113": "APC",
}

# Map the labels
dataset.map_labels(
    "ground_truth",
    label_mapping
).save()

### Evaluate a pretrained model on this dataset

Once our test dataset is ready, we can evaluated a pretrained model on it.

In [ ]:
from adomvi.yolo.yolo import predict

model = Path() / "runs/detect/train/weights/best.pt"
results_predict = predict(best_model, source=search_2_dir / "search_2/images")

# Load the path of the prediction model results
results_predict_dir = Path(results_predict[0].save_dir)

Let's load the model's predictions into our dataset.

In [ ]:
from adomvi.yolo.utils import add_yolo_detections

prediction_field = "yolov8"
predictions_dir = Path() / results_predict_dir / "labels"
add_yolo_detections(dataset, prediction_field=prediction_field, predictions_dir=predictions_dir, class_list=["AFV", "APC", "MEV", "LAV"])